### Modules used in this notebook
pypsa, plotnine

## E.002: Creating your first power system model
In this notebook you will create a simple power system model simulating a system with specific month of the year using wind and solar profiles based on atmospheric reanalysis data. The model will be optimised to define the optimal capacity of batteries needed to balance the system.

# Step 01: Setting up the environment
This part of the notebook install the needed Python modules and the solver ([cbc](https://github.com/coin-or/Cbc), a widely-used open source solver).
The installed Python modules are:
  - PyPSA: [an open-source power system model](https://pypsa.org/)

In [ ]:
!pip install plotnine
!pip install pypsa # power system model
!apt-get install -y -qq coinor-cbc # open source solver

# Step 02: Read wind, solar, demand data
Here the download a set of Parquet files containing the electricity demand, wind and solar profiles based on [this dataset](https://researchdata.reading.ac.uk/272/) developed by Bloomfield et al., University of Reading.

The Parquet files provide a data frame with the hourly values for all the European countries for the years 2000-2019 (the original dataset goes back to 1979 but we have kept only the last 20 years to reduce the amount of storage needed to run this notebook).

In [ ]:
import pandas as pd
dem = pd.read_parquet('../data/ERA5_full_demand_all_countries_2000_2019_hourly.parquet')
pv  = pd.read_parquet('../data/ERA5_solar_power_capacity_factor_all_countries_2000_2019_hourly.parquet')
win = pd.read_parquet('../data/ERA5_wind_power_capacity_factor_all_countries_2000_2019_inclusive.parquet')
dem.head()

# Step 03: selecting year, month, and country
In this example we simulate a power system model consisting of a wind power farm, a solar plant and a lithium-ion battery using the electricity demand and the RES profiles (capacity factors) for a specific country and a specific month.

In [ ]:
selected_year = 2012 # 2000-2019
selected_month = 2 # 1 = January, 12 = December
country = 'Spain'

In [ ]:
# Select demand, solar (PV) and wind profiles
selected_dem = dem.loc[(pd.DatetimeIndex(dem['datetime']).year == selected_year) & (pd.DatetimeIndex(dem['datetime']).month == selected_month)][country]
selected_pv  = pv.loc[(pd.DatetimeIndex(pv['datetime']).year == selected_year) & (pd.DatetimeIndex(pv['datetime']).month == selected_month)][country]
selected_win = win.loc[(pd.DatetimeIndex(win['datetime']).year == selected_year) & (pd.DatetimeIndex(win['datetime']).month == selected_month)][country]

# Step 04: Building the power system model

*   List item
*   List item


We create here a single-bus PyPSA network with:
  - `Wind`: A wind generator with the cost of 35 000 EUR per MW
  - `Solar`: a solar generator with the cost of 20 000 EUR per MW
  - `Battery`: a battery with 3 hours of storage with the cost of 45 000 EUR per MW. The efficiency is 90%.

The system has an electricity demand defined by the load `L1`

In [ ]:
import pypsa, logging
pypsa.pf.logger.setLevel(logging.WARNING)

network = pypsa.Network(snapshots = selected_dem.index)
network.add("Bus", "B1")

network.add("Generator", "Wind",
            carrier = 'wind',
            bus="B1",
            capital_cost = 35_000,
            p_max_pu = selected_win,
            p_nom_extendable = True,
            control="PQ")

network.add("Generator", "Solar",
            carrier = 'solar',
            bus="B1",
            capital_cost = 20_000,
            p_max_pu = selected_pv,
            p_nom_extendable = True,
            control="PQ")

network.add("StorageUnit", "Battery",
            bus="B1",
            capital_cost = 45_000,
            efficiency_store = 0.9,
            efficiency_dispatch = 0.9,
            max_hours = 3,
            p_nom_extendable = True
            )

network.add("Load", "L1",
            bus="B1",
            p_set=selected_dem*1e3)


In [ ]:
# prompt: A Pandas data frame containing in one column the values from selected_dem and in the column "hour" the values from 0 to 23 repeating 30 times.

import pandas as pd
flex_df = pd.DataFrame(
    selected_dem
)
flex_df['hour'] = list(range(24))*int(len(selected_dem)/24)
flex_df['e_min_pu'] = 0.5
flex_df.loc[(flex_df.hour > 9) & (flex_df.hour < 19), 'e_min_pu'] = 0
flex_df['e_max_pu'] = 0.5
flex_df.loc[(flex_df.hour > 9) & (flex_df.hour < 19), 'e_max_pu'] = 1
flex_df.head(4)


In [ ]:
FLEX_AMOUNT = (selected_dem.max() * 1e3 * 0.1) * 2
network.add(
    "Bus", "Bus_B1_DSR"
)
network.add(
    "Store", "B1_DSR",
    bus = "Bus_B1_DSR",

    e_nom = FLEX_AMOUNT * 2,
    e_initial = FLEX_AMOUNT,
    e_min_pu = flex_df['e_min_pu'],
    e_max_pu = flex_df['e_max_pu']

)
network.add(
    "Link", "Link_B1_DSR",
    bus0 = "B1",
    bus1 = "Bus_B1_DSR",
    p_nom = FLEX_AMOUNT,
    p_min_pu = -1
)

# Step 05: Running the model
Here we solve the linear optimal power flow defined in the PyPSA network created in the step before. We print the optimal capacities (in MW) of wind, solar and batteries found by the solver.

In [ ]:
network.optimize(solver_name = 'cbc')

print(round( network.generators.p_nom_opt ))
print(round(network.storage_units.p_nom_opt))

In [ ]:
network.statistics.expanded_capacity()

# Step 06: Plotting the dispatching
To plot the generation of the three assets modeled in this example we need to insert the data contained in the `network` object into a Pandas data frame. This is eventually used for a plot made with [plotnine](https://plotnine.readthedocs.io/en/stable/)

In [ ]:
gen_list = network.generators[['bus']].reset_index()
gen_list['type'] = gen_list['Generator'].str.split('_', expand = True).iloc[:,0]
gen = (pd.merge(
    network.generators_t.p
    .unstack()
    .reset_index(), gen_list)
       .groupby(['type', 'bus', 'snapshot'])
       .sum()
       .reset_index()
      ).rename(columns = {0: 'prod'})

sto_list = network.storage_units[['bus']].reset_index()
sto_list['type'] = sto_list['StorageUnit'].str.split('_', expand = True).iloc[:,0]
sto = (pd.merge(
    network.storage_units_t.p
    .unstack()
    .reset_index(), sto_list)
       .groupby(['type', 'bus', 'snapshot'])
       .sum().reset_index()
      ).rename(columns = {0: 'prod'})

df = pd.concat([gen, sto])
df.head()

In [ ]:
import plotnine
plotnine.options.figure_size = (12, 4)

(
    plotnine.ggplot(pd.DataFrame(network.loads_t.p_set.L1).reset_index(), plotnine.aes(x='snapshot', y='L1')) +
    plotnine.geom_area(plotnine.aes(x = 'snapshot', y = 'prod', fill = 'type'), data= df) +
    plotnine.geom_line()
)

In [ ]:
network.links_t.p0[0:168].plot()